In [ ]:
%load_ext cython

# Case Study: Slow Pandas dates

Batches of data are collected from field instruments. These instruments capture the date in **three separate columns**: day, month and year.

Data is processed in **Pandas**, but currently it is <u>slow to convert the three columns into datetimes</u>.

# Example (randomised) data

In [ ]:
import numpy as np
import pandas as pd

def make_sample_data(size):
    d = dict(
        # Years: 1980 - 2015
        year=np.random.randint(1980, 2016, int(size)),
        # Months 1 - 12
        month=np.random.randint(1, 13, int(size)),
        # Day number: 1 - 28
        day=np.random.randint(1, 28, int(size)),
        )
    return pd.DataFrame(d)

## Start with a small dataset

In [ ]:
df = make_sample_data(5)
df

## Goal: make single `datetime` column

Let's see the Python code first:

In [ ]:
import datetime

def create_datetime_py(year, month, day):
    """ Take year, month, day and return a datetime """
    return datetime.datetime(year, month, day, 0, 0, 0, 0, None)

## Use the Python conversion function

Pandas has an `apply()` method that runs your function on a bunch of columns.

You must provide a function that receives a **row**, and your function must return a value.  All the output values get put into a new Pandas **series**.

In [ ]:
# Refer to fields by name! Very cool 👍
df.apply(lambda x : create_datetime_py(
        x['year'], x['month'], x['day']), axis=1)

**Note: the type is "datetime64[ns]".**

Awkward to type that all out each time.  Let's make a convenient function.

In [ ]:
def make_datetime_py(df):
    return df.apply(lambda x : create_datetime_py(
        x['year'], x['month'], x['day']), axis=1)

Then we can just call it like so:

In [ ]:
df['new'] = make_datetime_py(df)
df

# Problem: this is slow

With lots of data, the conversion to a datetime column takes a very long time!  Let's try a bunch of data:

In [ ]:
df_big = make_sample_data(100000)

%timeit make_datetime_py(df_big)

# What to do?

The first thing is to check whether there is a low-level PXD interface file for the Python `datetime` object.

![title](cython-includes-github.png)

# Let's use Cython!

In [ ]:
%%cython -a
# cython: boundscheck = False
# cython: wraparound = False
from cpython.datetime cimport (
    import_datetime, datetime_new, datetime, timedelta)
from pandas import Timestamp

import_datetime()

cpdef convert_arrays_ts(
        long[:] year, long[:] month, long[:] day, 
        long long[:] out):
    """ Result goes into `out`  """
    cdef int i, n = year.shape[0]
    cdef datetime dt
    for i in range(n):
        dt = <datetime>datetime_new(
                year[i], month[i], day[i], 0, 0, 0, 0, None)
        out[i] = Timestamp(dt).value

### Utility function for applying our conversion

In [ ]:
def make_datetime_cy(df, method):
    s = pd.Series(np.zeros(len(df), dtype='datetime64[ns]'))
    method(df['year'].values, df['month'].values, df['day'].values,
               s.values.view('int64')) 
    return s

In [ ]:
# Test it out
make_datetime_cy(df, convert_arrays_ts)

## Speed Test

In [ ]:
df_big = make_sample_data(100000)

%timeit make_datetime_py(df_big)
%timeit make_datetime_cy(df_big, convert_arrays_ts)

- Check annotation

# Eliminate the Python overhead

In [ ]:
%%cython -a
# cython: boundscheck = False
# cython: wraparound = False
from cpython.datetime cimport (
    import_datetime, datetime_new, datetime, timedelta,
    timedelta_seconds, timedelta_days)

import_datetime()  # <-- Pretty important

cpdef convert_arrays_dt(long[:] year, long[:] month, long[:] day, 
        long long[:] out):
    """ Result goes into `out`  """
    cdef int i, n = year.shape[0]
    cdef datetime dt, epoch = datetime_new(1970, 1, 1, 0, 0, 0, 0, None)
    cdef timedelta td
    cdef long seconds
    for i in range(n):
        dt = <datetime>datetime_new(
                year[i], month[i], day[i], 0, 0, 0, 0, None)
        td = <timedelta>(dt - epoch)
        seconds = timedelta_days(td) * 86400 # Seconds per day
        out[i] = seconds * 1000000000  # Nanoseconds, remember?

# Test it out

In [ ]:
make_datetime_cy(df, convert_arrays_dt)

# Speed Test

In [ ]:
df_big = make_sample_data(100000)

%timeit make_datetime_py(df_big)
%timeit make_datetime_cy(df_big, convert_arrays_ts)
%timeit make_datetime_cy(df_big, convert_arrays_dt)

<div style="margin-top: 200px;"></div>
# Speed improvement:
<p style="font-size: 64px;">~850 X !!!</p>
<div style="margin-top: 40px;"></div>
- real Python datetime and timedelta objects
- still single-threaded